

## **Chapter 15: Federation and Microservices**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Evaluate the trade-offs between monolithic and microservices architectures for GraphQL
- Understand the Apollo Federation architecture (Gateway and Subgraphs)
- Define Entities using the `@key` directive to enable cross-service references
- Implement the `@external`, `@requires`, and `@provides` directives for advanced service boundaries
- Compose multiple subgraph schemas into a unified supergraph
- Migrate an existing monolith to a federated architecture incrementally
- Handle authentication and authorization across distributed services
- Debug and monitor a federated graph using Apollo Studio

---

## **Prerequisites**

- Completed Chapters 7-14 (Server implementation through Testing)
- Understanding of microservices concepts (service boundaries, inter-service communication)
- Familiarity with Docker/containerization (helpful but not required)
- Basic knowledge of load balancing and reverse proxies

---

## **15.1 The Monolith vs. Microservices Dilemma**

As your GraphQL API grows, you face a critical architectural decision: keep it as a monolith or split it into microservices?

### **The Monolithic GraphQL Server**

**Architecture:**
```
Client → Single GraphQL Server → Multiple Databases/APIs
                ↓
        [Resolvers for Users, Orders, Products, Reviews]
```

**Pros:**
- Simple deployment and testing
- Easy refactoring across domains
- No network overhead between resolvers
- Single codebase to maintain

**Cons:**
- Team coordination bottlenecks (multiple teams touching one codebase)
- Technology lock-in (must use same language/framework for everything)
- Deployment risk (one bug takes down the entire API)
- Scaling inefficiencies (must scale entire app, not just hot paths)

### **The Microservices Approach**

**Architecture:**
```
Client → API Gateway → User Service → User DB
                ↓
         Order Service → Order DB
                ↓
       Product Service → Product DB
```

**The Problem:** In REST, each service has its own endpoints. In GraphQL, the client expects a single, unified graph. How do we maintain "one graph" while having "many services"?

**The Solution: Apollo Federation**

Apollo Federation allows you to maintain a single GraphQL schema (the "supergraph") while distributing its implementation across multiple services (subgraphs). Each team owns their subgraph, but clients see one unified API.

---

## **15.2 Introduction to Apollo Federation**

Apollo Federation is an architecture for composing multiple GraphQL services (subgraphs) into a unified graph (supergraph).

### **15.2.1 The Gateway Pattern**

The **Gateway** (or Router in Apollo Federation 2) is the entry point for all client requests. It doesn't contain business logic; instead, it:
1. Receives the incoming query
2. Analyzes which subgraphs can fulfill each part
3. Sends optimized sub-queries to each subgraph
4. Merges the results into a single response

**Architecture:**
```
Client → Gateway/Router → Subgraph A (Users)
                  ↓
           Subgraph B (Orders)
                  ↓
           Subgraph C (Products)
```

**Gateway Implementation:**

```javascript
const { ApolloGateway, IntrospectAndCompose } = require('@apollo/gateway');
const { ApolloServer } = require('apollo-server');

const gateway = new ApolloGateway({
  supergraphSdl: new IntrospectAndCompose({
    subgraphs: [
      { name: 'users', url: 'http://localhost:4001/graphql' },
      { name: 'orders', url: 'http://localhost:4002/graphql' },
      { name: 'products', url: 'http://localhost:4003/graphql' },
    ],
  }),
});

const server = new ApolloServer({
  gateway,
  // Disable subscriptions in gateway mode (use router for Fed 2)
  subscriptions: false,
});

server.listen().then(({ url }) => {
  console.log(`🚀 Gateway ready at ${url}`);
});
```

### **15.2.2 Implementing a Federated Service (Subgraph)**

Each subgraph is a standalone Apollo Server that contributes part of the schema.

**User Subgraph (`users.js`):**

```javascript
const { ApolloServer, gql } = require('apollo-server');
const { buildSubgraphSchema } = require('@apollo/subgraph');

const typeDefs = gql`
  # The @key directive marks this type as an entity
  # External services can reference a User by its id
  type User @key(fields: "id") {
    id: ID!
    name: String!
    email: String!
  }

  type Query {
    me: User
    users: [User!]!
  }
`;

const resolvers = {
  Query: {
    me: () => ({ id: '1', name: 'Alice', email: 'alice@example.com' }),
    users: () => [{ id: '1', name: 'Alice', email: 'alice@example.com' }],
  },
  
  // Required for federation: __resolveReference
  User: {
    __resolveReference(user, { dataSources }) {
      // Fetch the full user when referenced by another service
      return dataSources.userAPI.getUserById(user.id);
    },
  },
};

const server = new ApolloServer({
  schema: buildSubgraphSchema({ typeDefs, resolvers }),
  context: () => ({ /* auth, data sources */ }),
});

server.listen({ port: 4001 }).then(({ url }) => {
  console.log(`👤 Users service ready at ${url}`);
});
```

**Key Differences from Monolith:**
1. Use `buildSubgraphSchema` instead of standard schema
2. Define `@key` directives on types that other services need to reference
3. Implement `__resolveReference` for entity types (federation's way of fetching an object by its key)

---

## **15.3 Key Concepts: Entities, Keys, and References**

### **Entities**

An **Entity** is a type that can be referenced by other subgraphs. It's defined using the `@key` directive, which specifies which fields uniquely identify the object.

```graphql
type User @key(fields: "id") {
  id: ID!
  name: String!
}
```

**Compound Keys:**
You can use multiple fields as a composite key:

```graphql
type Organization @key(fields: "id") @key(fields: "name") {
  id: ID!
  name: String!
  industry: String!
}
```

### **Referencing Entities (The @external Directive)**

When one service needs to extend a type from another service, it uses `@external` to declare that it doesn't own that field.

**Orders Subgraph extending User:**

```javascript
const typeDefs = gql`
  # Extend the User type from the Users subgraph
  extend type User @key(fields: "id") {
    id: ID! @external  # We don't own this, but we need it as a key
    orders: [Order!]!  # This is the field we're adding
  }

  type Order @key(fields: "id") {
    id: ID!
    buyer: User!       # Reference to the User entity
    total: Float!
  }

  type Query {
    order(id: ID!): Order
  }
`;

const resolvers = {
  Order: {
    __resolveReference(order) {
      return fetchOrderById(order.id);
    },
    
    // Resolver for the buyer field
    buyer(order) {
      // Return a reference object
      // The gateway will fetch the actual user from the Users subgraph
      return { __typename: "User", id: order.buyerId };
    },
  },
  
  User: {
    // Resolve orders for a user when requested
    orders(user) {
      return fetchOrdersByUserId(user.id);
    },
  },
};
```

**Explanation:**
- `extend type User`: Declares that User is defined elsewhere
- `@external`: Marks `id` as owned by another service
- Returning `{ __typename: "User", id: user.id }`: Creates a "reference" that the gateway uses to stitch data together

---

## **15.4 Federation Directives**

Federation 2 introduces several directives to define service boundaries clearly.

### **@key**
Marks a type as an entity and specifies its primary key(s).

```graphql
type Product @key(fields: "sku") @key(fields: "upc") {
  sku: String!
  upc: String!
  name: String!
}
```

### **@external**
Indicates a field is defined in another subgraph. Used when you need to reference a field but don't provide its data.

```graphql
extend type Product @key(fields: "sku") {
  sku: String! @external
  inStock: Boolean!  # Local field
  shippingEstimate: Int @requires(fields: "weight")  # Needs weight from other service
}
```

### **@requires**
Indicates that resolving a field requires data from the entity's other fields (potentially from other subgraphs).

```graphql
extend type Product @key(fields: "sku") {
  sku: String! @external
  weight: Float @external  # From Products subgraph
  
  # To calculate shipping cost, we need the weight
  shippingCost: Float @requires(fields: "weight")
}
```

**Implementation:**
```javascript
const resolvers = {
  Product: {
    shippingCost(product) {
      // product.weight is automatically provided by the gateway
      // because of @requires(fields: "weight")
      return calculateShipping(product.weight);
    },
  },
};
```

### **@provides**
Indicates that a field can be resolved by this subgraph, even if the type is primarily owned elsewhere. Useful for optimization (avoiding extra round-trips).

```graphql
type Review {
  id: ID!
  product: Product! @provides(fields: "name")  # We can provide the name
  rating: Int!
}

extend type Product @key(fields: "id") {
  id: ID! @external
  name: String @external  # Reviews subgraph can provide this when queried via Review.product
}
```

---

## **15.5 Managing Cross-Service Communication**

In a federated architecture, services often need to communicate. There are two patterns:

### **Pattern 1: Gateway Orchestration (Preferred)**
The Gateway handles fetching data from multiple services and composing the response. Your subgraphs remain simple and don't call each other directly.

**Pros:** Loose coupling, better caching, easier to reason about
**Cons:** Multiple network hops

### **Pattern 2: Subgraph-to-Subgraph Communication**
A subgraph directly calls another subgraph's API when needed.

```javascript
// In Orders subgraph, calling Users subgraph
const resolvers = {
  Order: {
    async buyer(order, _, { dataSources }) {
      // Direct HTTP call to Users service
      // Only do this when necessary (e.g., for performance)
      return dataSources.usersAPI.getUser(order.buyerId);
    },
  },
};
```

**When to use:** When the Gateway's automatic resolution would be too slow or when you need transactional consistency across services (rare in GraphQL).

---

## **15.6 Schema Composition and Merging**

Federation automatically composes subgraph schemas into a supergraph. However, conflicts can occur.

### **Conflict Resolution Rules**

1. **Value Types:** Types without @key must be identical across all subgraphs
   ```graphql
   # Both subgraphs must define Date the same way
   scalar Date
   ```

2. **Entity Ownership:** Only one subgraph should define the base fields of an entity. Others extend it.

3. **Field Conflicts:** If two subgraphs define the same field on a type, they must have compatible types.

### **Rover CLI for Schema Management**

Use Apollo Rover to check composition before deployment:

```bash
# Install Rover
npm install -g @apollo/rover

# Check if subgraphs compose correctly
rover supergraph compose --config supergraph.yaml
```

**supergraph.yaml:**
```yaml
federation_version: 2
subgraphs:
  users:
    routing_url: http://localhost:4001
    schema:
      file: ./users.graphql
  orders:
    routing_url: http://localhost:4002
    schema:
      file: ./orders.graphql
```

**CI/CD Integration:**
```bash
# In your deployment pipeline
rover subgraph check my-graph@production \
  --schema ./users.graphql \
  --name users
```

---

## **15.7 Migrating to Federation: A Step-by-Step Guide**

Migrating a monolith to federation should be incremental to minimize risk.

### **Phase 1: Strangler Fig Pattern**
Move one domain at a time from the monolith to a new subgraph.

**Step 1:** Identify a bounded context (e.g., "Reviews")
**Step 2:** Create Reviews subgraph with just the schema
**Step 3:** Monolith delegates Reviews queries to the new subgraph
**Step 4:** Once stable, cut over traffic to the subgraph

### **Phase 2: Entity Extraction**
Extract entities that are referenced by multiple services.

**Example: Extracting User from Monolith**

1. **Create Users Subgraph:**
   ```graphql
   type User @key(fields: "id") {
     id: ID!
     name: String!
     email: String!
   }
   ```

2. **Update Monolith (now Order Subgraph):**
   ```graphql
   extend type User @key(fields: "id") {
     id: ID! @external
     orders: [Order!]!  # Keep orders here
   }
   ```

3. **Gradually move user-related mutations to Users subgraph**

### **Phase 3: Router Cutover**
Replace the monolith's GraphQL endpoint with the Gateway, keeping monolith as one big subgraph initially.

```
Before: Client → Monolith

After:  Client → Gateway → Monolith (as subgraph)
                ↘ New Services
```

Then slowly break pieces off the monolith-subgraph into separate services.

---

## **Chapter Summary**

Apollo Federation enables you to scale GraphQL across teams and services while maintaining the "one graph" philosophy that makes GraphQL powerful.

### **Key Takeaways:**

1.  **Federation vs. Monolith:** Use federation when multiple teams need to own different parts of the schema independently, or when you need service-specific scaling.
2.  **Entities:** Types marked with `@key` can be referenced across service boundaries. They are the glue of your supergraph.
3.  **Subgraph Responsibilities:** Each subgraph owns specific fields. Use `@external` to use fields owned by others, `@requires` to fetch dependencies, and `@provides` to optimize resolution.
4.  **Gateway:** The entry point that orchestrates queries across subgraphs. It handles the hard work of query planning and execution.
5.  **Migration:** Use the Strangler Fig pattern. Move one domain at a time. Keep the monolith as a large subgraph initially, then decompose.
6.  **Schema Checks:** Always use Rover or similar tools to verify subgraphs compose correctly before deployment.

### **Federation Checklist:**

- [ ] Define clear domain boundaries for each subgraph
- [ ] Mark shared types as Entities with `@key`
- [ ] Implement `__resolveReference` for all entities
- [ ] Use `@external` when referencing fields from other subgraphs
- [ ] Configure proper authentication/authorization at the gateway level
- [ ] Set up distributed tracing across subgraphs
- [ ] Use Rover CLI for schema composition checks in CI/CD
- [ ] Monitor subgraph health independently
- [ ] Plan for breaking changes across multiple services carefully

---

### **🚀 Next Up: Chapter 16 - Tooling and Developer Experience**

**Summary:** We have mastered the architecture of large-scale GraphQL systems. In Chapter 16, we will explore the tooling ecosystem that makes development productive and enjoyable. You will learn about code generation (GraphQL Code Generator) for type-safe clients, schema linting and governance, documentation best practices, and VS Code extensions that supercharge your GraphQL workflow.**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../5. production_grade_engineering/14. testing_graphql.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='16. tooling_and_developer_experience.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
